In [1]:
import os 
from dotenv import load_dotenv
os.chdir("..")
load_dotenv()

True

In [2]:
#sql tables
#rent table includes information about Investment rent_collection_sample_with_historic_payments
#lease table includes tenant_propery_owner_lease_rent_details

rent_table = "rent_table"
lease_table = "lease_table"


In [5]:
from langchain_community.utilities import SQLDatabase

database = "real_estate.db"
sqlite_url = f"sqlite:///{database}"

db = SQLDatabase.from_uri(sqlite_url)
print(db.dialect)
print(db.get_usable_table_names())
db.run(f"SELECT * FROM {rent_table} LIMIT 10;")

sqlite
['lease_table', 'rent_table']


"[('John Smith', '15C', '13-12-2023', '15-12-2024', '13-12-2023', 'On time', 0, 'john.smith@example.com', 90001), ('John Smith', '15C', '13-12-2023', '15-12-2024', '12-01-2024', 'On time', 0, 'john.smith@example.com', 90001), ('John Smith', '15C', '13-12-2023', '15-12-2024', '11-02-2024', 'Delayed', 8, 'john.smith@example.com', 90001), ('John Smith', '15C', '13-12-2023', '15-12-2024', '12-03-2024', 'Delayed', 7, 'john.smith@example.com', 90001), ('John Smith', '15C', '13-12-2023', '15-12-2024', '11-04-2024', 'Delayed', 5, 'john.smith@example.com', 90001), ('John Smith', '15C', '13-12-2023', '15-12-2024', '11-05-2024', 'Delayed', 8, 'john.smith@example.com', 90001), ('John Smith', '15C', '13-12-2023', '15-12-2024', '10-06-2024', 'Delayed', 6, 'john.smith@example.com', 90001), ('John Smith', '15C', '13-12-2023', '15-12-2024', '10-07-2024', 'On time', 0, 'john.smith@example.com', 90001), ('John Smith', '15C', '13-12-2023', '15-12-2024', '09-08-2024', 'Delayed', 4, 'john.smith@example.com'

In [4]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
sql_agent = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [5]:
sql_agent.invoke({"input": "what is the monthly rent for John Smith, Apartment 15C."})





> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


lease_table, rent_table
Invoking: `sql_db_schema` with `{'table_names': 'lease_table'}`



CREATE TABLE lease_table (
	tenant_name TEXT, 
	apartment_number TEXT, 
	email TEXT, 
	tenant_phone INTEGER, 
	tenant_id INTEGER, 
	rental_amount INTEGER, 
	payment_date TEXT, 
	due_date TEXT, 
	status TEXT, 
	city TEXT, 
	apartment_building TEXT, 
	lease_start_date TEXT, 
	lease_end_date TEXT, 
	lease_terms_conditions TEXT, 
	renewal_terms_conditions TEXT, 
	owner_name TEXT, 
	owner_email TEXT, 
	owner_contact_number TEXT
)

/*
3 rows from lease_table table:
tenant_name	apartment_number	email	tenant_phone	tenant_id	rental_amount	payment_date	due_date	status	city	apartment_building	lease_start_date	lease_end_date	lease_terms_conditions	renewal_terms_conditions	owner_name	owner_email	owner_contact_number
John Smith	15C	john.smith@example.com	987654321	1	1200	01-12-2024	05-12-2024	Paid	Los Angeles	Riverside Tow

{'input': 'what is the monthly rent for John Smith, Apartment 15C.',
 'output': 'The monthly rent for John Smith in Apartment 15C is $1200.'}

In [6]:
system_prompt = """You are an AI agent designed to assist tenants with lease management requests. Your goals are:

### Core Responsibilities
1. **For Lease Renewal**:
   - Confirm the tenant's details (name, apartment number).
   - Retrieve the lease details from the database (e.g., expiration date, rental amount). 
   - Share the lease details with expiration dates and monthly rent to be paid. 
   - Confirm from the tenant if they wanna proceed with the renewal
   - Generate a lease renewal agreement as a PDF if they agrees to renew. 
   - Send an email with the subject "Lease Renewal Agreement" to the tenant, attaching the PDF agreement. Include a polite message requesting review and signature.
   - Update the system records after the tenant confirms signing the lease.

2. **For Rent Payment**:
   - Confirm the tenant's details (name, apartment number).
   - Check if rent is paid for this month or not from the database. 
   - If not paid, show the user details about the payment, last date. 
   _ If confirms to pay the rent, proceed with sending email with payment link.
   - Use the constant payment link `abc.xyz@xyzbank` to compose a rent payment email.
   - Send an email with the subject "Rent Payment Details" and include the payment link in the body with appropriate greetings.

3. **Error Handling**:
   - If a tool (e.g., email sender or lease generator) encounters an error, inform the tenant politely. For example, "We are experiencing technical difficulties. Please try again shortly."
   - Log all errors for further review.

4. **Fallback Actions**:
   - If the tenant's request is unclear, ask clarifying questions to determine the scenario (e.g., "Are you looking to renew your lease or make a rent payment?").
   - If a tool is unavailable, suggest an alternative. For example, "You can manually renew your lease by visiting our office or contacting support."

5. **Communication Style**:
   - Maintain a professional, clear, and polite tone.
   - Ensure all responses are concise and action-oriented.
   - Acknowledge tenant actions promptly (e.g., "Thank you for renewing your lease!").

### Available Tools
1. **SQLite Lease Database Tool**: Retrieve tenant and lease details.
2. **Lease Agreement Generator**: Create PDF files for lease agreements.
3. **Email Sender Tool**: Send emails with attachments or a constant payment link (`abc.xyz@xyzbank`).

### Execution Guidelines
- Assess the tenant's request (lease renewal or rent payment) based on the conversation context.
- Use the appropriate tools for the identified scenario.
- Always confirm successful execution of tasks with the tenant.
- Be prepared to escalate unresolved issues or unusual requests.

### Notes for Scalability
- The current payment link is constant. For future changes, refer to the system database or configuration files.
- Log all interactions to improve future automation and error resolution.

"""

In [7]:
## tools

from typing import Optional
from langchain_core.tools import tool
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from fpdf import FPDF


@tool
def sql_agent_tool(input):
    """Use this tool to interact with the SQLite database to retrieve tenant and lease data.
    This tool could be used to do all the database related activities including CRUD. 

    """
    return sql_agent.invoke({"input": input})


@tool
def generate_lease_agreement_tool(tenant_name: str, 
    apartment_number: str, 
    tenant_email: str, 
    tenant_phone: str, 
    owner_name: str, 
    owner_email: str, 
    owner_contact: str, 
    property_name: str, 
    city: str, 
    zip_code: str, 
    lease_start: str, 
    lease_end: str, 
    rent_amount: str) -> str:
    """Generates a professional lease agreement based on tenant and lease data."""
    try:
        

        agreement = f"""
        LEASE AGREEMENT
        Tenant Name: {tenant_name}
        Tenant Apartment: {apartment_number}
        Tenant Email: {tenant_email}
        Tenant Contact: {tenant_phone}

        Owner Name: {owner_name}
        Owner Email: {owner_email}
        Owner Contact: {owner_contact}

        Property: {property_name}
        Property City: {city}
        Property Zip Code: {zip_code}

        Basic Terms and Conditions:
        1. Lease Term: Start Date: {lease_start}, End Date: {lease_end}
        2. Rent: ${rent_amount} per month, due on the 5th of each month. Payment methods include cheque or online payment.
        3. Security Deposit: A security deposit of ${rent_amount} is required and refundable subject to property condition.
        4. Maintenance and Repairs: Tenant handles minor repairs; landlord handles major repairs unless caused by tenant negligence.
        5. Utilities: Tenant pays for electricity, water, and gas.
        6. Default: Landlord may terminate the lease and seek damages for breaches or non-payment of rent.
        7. Governing Law: This lease is governed by the laws of the property location.

        Renewal Terms:
        1. Automatic renewal unless a 30-day notice is given.
        2. Rent increases by 5% annually.
        3. Renewal lease term is one year unless agreed otherwise.

        Signed By:
        Tenant Signature: _____________________   Date: ___________
        Owner Signature: ______________________   Date: ___________
        """
        
        # Initialize PDF
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        
        # Add content to the PDF
        for line in agreement.strip().split("\n"):
            pdf.cell(200, 10, txt=line.strip(), ln=True)

        # Define file path
        file_name = f"Lease_Agreement_{tenant_name.replace(' ', '_')}.pdf"
        file_path = os.path.join("generated_leases", file_name)
        
        # Ensure the directory exists
        os.makedirs("generated_leases", exist_ok=True)

        # Save PDF
        pdf.output(file_path)

        return file_path
    except Exception as e:
        return f"Error generating lease agreement: {str(e)}"
    
    
    


@tool
def send_email_tool(
    tenant_first_name:str,
    recipient_email: str,
    subject: str,
    attachment_path: Optional[str] = None,
    include_payment_link: bool = False,
) -> str:
    
    
    """Send an email with optional attachment and payment link.

    Args:
        recipient_email (str): Email address of the recipient
        subject (str): Subject line of the email
        email_body (str): Main body of the email
        attachment_path (str, optional): Path to the file to be attached
        include_payment_link (bool, optional): Whether to include a payment link
        payment_link (str, optional): Payment link to be included in the email

    Returns:
        str: Confirmation message about email sending status
    
    
    """
    
    try:
        # Retrieve email credentials from environment variables
        sender_email = os.getenv("SENDER_EMAIL")
        sender_password = os.getenv("SENDER_APP_PASSWORD")

        if not sender_email or not sender_password:
            raise ValueError("Email credentials not found in environment variables")

        # SMTP server configuration
        smtp_server = "smtp.gmail.com"
        smtp_port = 587

        # Compose email
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = subject

        greeting_body = f"""Hi {tenant_first_name},\nThank you for contacting us. """
        # Prepare email body
        if include_payment_link:
            email_body = greeting_body+"PLease follow below link for payment -   abc.xyz@Pqr"

            msg.attach(MIMEText(email_body, 'plain'))
        else:
            email_body = greeting_body+"Please find the lease agreement below."
            msg.attach(MIMEText(email_body, 'plain'))

            # Attach file if provided
            if attachment_path and os.path.exists(attachment_path):
                try:
                    with open(attachment_path, 'rb') as file:
                        part = MIMEBase('application', 'octet-stream')
                        part.set_payload(file.read())
                        encoders.encode_base64(part)
                        part.add_header(
                            'Content-Disposition', 
                            f'attachment; filename="{os.path.basename(attachment_path)}"'
                        )
                        msg.attach(part)
                except Exception as attach_error:
                    print(f"Error attaching file: {attach_error}")
                # Continue sending email even if attachment fails

        # Send email
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.send_message(msg)
            print(f"Email sent successfully to {recipient_email}")

        return f"Email successfully sent to {recipient_email}"

    except smtplib.SMTPException as smtp_error:
        print(f"SMTP error occurred: {smtp_error}")
        return f"Failed to send email: SMTP error - {smtp_error}"
    except Exception as error:
        print(f"Unexpected error occurred: {error}")
        return f"Failed to send email: Unexpected error - {error}"

    


In [ ]:
#test email
# Demo input data for Rent Payment
    
recipient_email = "hrisikesh.neogi@gmail.com"
subject= "Rent Payment Reminder"
message= "Dear Tenant, your rent is due. Please make your payment using the link below."
payment_link= "abc.xyz@xyzbank"

# Call the tool to send the email
response = send_email_tool(
    tenant_first_name="Hrisikesh",
    recipient_email=recipient_email,
    attachment_path="research/Lease Renewal Scenario -pdf.pdf",
    subject="Your requested document",
    include_payment_link=True  # Set to True or False based on your need
)    


# Output the response (success message)
print(response)


Email sent successfully to hrisikesh.neogi@gmail.com
Email successfully sent to hrisikesh.neogi@gmail.com


In [8]:
tools = [sql_agent_tool,
         generate_lease_agreement_tool,
         send_email_tool
         ]


In [9]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history",
                                  return_messages=True)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools,
    verbose=False,
    memory=memory  # You might want to add memory here
)

/var/folders/wk/gz11kjt56mj4rq6n_cbs66540000gn/T/ipykernel_66749/1202805409.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history",


In [10]:
#lease renewal agent
#validate the user details using some otp or mail
while True:
    query = input("Ask a question:")
    output = agent_executor.invoke({"input": query})
    print("Input:", output['input'])
    print("Output:", output['output'])
    if query.lower() == "exit":
        break




Ask a question: Hi, I want to renew my lease.


Input: Hi, I want to renew my lease.
Output: Could you please provide me with your name and apartment number so I can assist you with the lease renewal?


Ask a question: John Smith	15C




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


lease_table, rent_table
Invoking: `sql_db_schema` with `{'table_names': 'lease_table'}`



CREATE TABLE lease_table (
	tenant_name TEXT, 
	apartment_number TEXT, 
	email TEXT, 
	tenant_phone INTEGER, 
	tenant_id INTEGER, 
	rental_amount INTEGER, 
	payment_date TEXT, 
	due_date TEXT, 
	status TEXT, 
	city TEXT, 
	apartment_building TEXT, 
	lease_start_date TEXT, 
	lease_end_date TEXT, 
	lease_terms_conditions TEXT, 
	renewal_terms_conditions TEXT, 
	owner_name TEXT, 
	owner_email TEXT, 
	owner_contact_number TEXT
)

/*
3 rows from lease_table table:
tenant_name	apartment_number	email	tenant_phone	tenant_id	rental_amount	payment_date	due_date	status	city	apartment_building	lease_start_date	lease_end_date	lease_terms_conditions	renewal_terms_conditions	owner_name	owner_email	owner_contact_number
John Smith	15C	john.smith@example.com	987654321	1	1200	01-12-2024	05-12-2024	Paid	Los Angeles	Riverside Tow

Ask a question: second one


...

Email sent successfully to john.smith@example.com
Input: yes
Output: Your lease renewal agreement has been successfully generated and sent to your email at john.smith@example.com. Please review the document and sign it at your earliest convenience.

Thank you for renewing your lease! If you have any further questions or need assistance, feel free to ask.


Ask a question: thanks I got it


Input: thanks I got it
Output: You're welcome! If you need any more assistance in the future, feel free to reach out. Have a great day!


Ask a question: exit


Input: exit
Output: Thank you for using our service! If you need assistance again, don't hesitate to return. Have a wonderful day!


In [ ]:
#rent payment agent - already paid

while True:
    query = input("Ask a question:")
    output = agent_executor.invoke({"input": query})
    print("Input:", output['input'])
    print("Output:", output['output'])
    if query.lower() == "exit":
        break




Ask a question: hi. I wanna pay the rent 


Input: hi. I wanna pay the rent 
Output: Could you please provide me with your name and apartment number so I can assist you with the rent payment?


Ask a question: John Smith	15C




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


lease_table, rent_table
Invoking: `sql_db_schema` with `{'table_names': 'lease_table'}`



CREATE TABLE lease_table (
	tenant_name TEXT, 
	apartment_number TEXT, 
	email TEXT, 
	tenant_phone INTEGER, 
	tenant_id INTEGER, 
	rental_amount INTEGER, 
	payment_date TEXT, 
	due_date TEXT, 
	status TEXT, 
	city TEXT, 
	apartment_building TEXT, 
	lease_start_date TEXT, 
	lease_end_date TEXT, 
	lease_terms_conditions TEXT, 
	renewal_terms_conditions TEXT, 
	owner_name TEXT, 
	owner_email TEXT, 
	owner_contact_number TEXT
)

/*
3 rows from lease_table table:
tenant_name	apartment_number	email	tenant_phone	tenant_id	rental_amount	payment_date	due_date	status	city	apartment_building	lease_start_date	lease_end_date	lease_terms_conditions	renewal_terms_conditions	owner_name	owner_email	owner_contact_number
John Smith	15C	john.smith@example.com	987654321	1	1200	01-12-2024	05-12-2024	Paid	Los Angeles	Riverside Tow

Ask a question: no thanks.


Input: no thanks.
Output: You're welcome! If you have any other questions in the future, feel free to reach out. Have a great day!


Ask a question: exit


Input: exit
Output: Thank you for using our service! If you need assistance again, don't hesitate to return. Have a wonderful day!


In [ ]:
#rent payment agent - rent unpaid

while True:
    query = input("Ask a question:")
    output = agent_executor.invoke({"input": query})
    print("Input:", output['input'])
    print("Output:", output['output'])
    if query.lower() == "exit":
        break




Ask a question: hi, I wanna pay the rent for this month


Input: hi, I wanna pay the rent for this month
Output: Could you please provide me with your name and apartment number so I can assist you with the rent payment?


Ask a question: Alice Brown	8A




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


lease_table, rent_table
Invoking: `sql_db_schema` with `{'table_names': 'lease_table'}`



CREATE TABLE lease_table (
	tenant_name TEXT, 
	apartment_number TEXT, 
	email TEXT, 
	tenant_phone INTEGER, 
	tenant_id INTEGER, 
	rental_amount INTEGER, 
	payment_date TEXT, 
	due_date TEXT, 
	status TEXT, 
	city TEXT, 
	apartment_building TEXT, 
	lease_start_date TEXT, 
	lease_end_date TEXT, 
	lease_terms_conditions TEXT, 
	renewal_terms_conditions TEXT, 
	owner_name TEXT, 
	owner_email TEXT, 
	owner_contact_number TEXT
)

/*
3 rows from lease_table table:
tenant_name	apartment_number	email	tenant_phone	tenant_id	rental_amount	payment_date	due_date	status	city	apartment_building	lease_start_date	lease_end_date	lease_terms_conditions	renewal_terms_conditions	owner_name	owner_email	owner_contact_number
John Smith	15C	john.smith@example.com	987654321	1	1200	01-12-2024	05-12-2024	Paid	Los Angeles	Riverside Tow

Ask a question: yes


Email sent successfully to alice.brown@example.com
Input: yes
Output: The rent payment details have been successfully sent to your email at alice.brown@example.com. Please check your inbox for the payment link.

Thank you for your prompt attention to your rent payment! If you need any further assistance, feel free to ask.


Ask a question: thanks. got it. made the payment. 


Input: thanks. got it. made the payment. 
Output: Thank you for confirming the payment! If you need any further assistance or have any questions, feel free to reach out. Have a great day!


Ask a question: exit


Input: exit
Output: Thank you for using our service! If you need assistance again, don't hesitate to return. Have a wonderful day!
